Premier essai basique avec un prompt inspiré de l'exercice pour générer du CSV (qui n'était pas complètement concluant). Transformation "manuelle" en JSON puis sauvegarde en JSON_L

A noter le problème des nombres de salles de bain qui est généralement un réel et non un entier.

-> prochain test avec l'utilisation de pydantic pour forcer le LLM à mieux respecter un format qui sera transformé grâce à des mécanismes de LangChain. 

In [1]:
import os
os.environ["OPENAI_API_KEY"] = "voc-8162499801266773377505669655d3c05508.40840521"
os.environ["OPENAI_API_BASE"] = "https://openai.vocareum.com/v1"

In [2]:
#from langchain_openai import OpenAI
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate

import re

In [3]:
num_ads = 10
advert_separator = "======="

In [4]:
model_name="gpt-3.5-turbo-instruct" #gpt-4
temperature = 0.0
llm = OpenAI(
    model_name=model_name, temperature=temperature, max_tokens=3000
)

/Users/me/opt/miniconda3/envs/udacity_cnn_project/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:141: LangChainDeprecationWarning: The class `OpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAI`.
  warn_deprecated(


In [5]:
# Note : impossible to get CSV formated rows 
#data_gen_template="""
#generate {num_ads} property advertisements for middle-class buyers. The advert contains the location in USA including the name the neighborhood, a detailed description including its surface area in square feet, the number of rooms, bedrooms and bathrooms, the number of floors, whether it has a garage and a garden, the style of construction and its price in dollars.
#each advert will have these 9 fields : neighborhood, style, rooms, bedrooms, bathrooms, house size, price, real estate description, neighborhood description.
#be creative in your descriptions but consistent and realistic. separate each advert with a string of seven '#' strictly.
#"""

In [6]:
# Note : impossible to get CSV formated rows 
data_gen_template="""
generate {num_ads} property advertisements for middle-class buyers. The advert contains the location in USA including the name the neighborhood, a detailed description including its surface area in square feet, the number of rooms, bedrooms and bathrooms, the number of floors, if there are a garage and a garden, the style of construction and its price in dollars.
each advert will have these 10 fields : advert number, neighborhood, style, rooms, bedrooms, bathrooms, house size, price, real estate description, neighborhood description.
be creative in your descriptions but consistent and realistic. the numbers of rooms, bedrooms and bathrooms must be integer values (not floating point). each advert must finish with the exact string '{advert_separator}'
"""

data_gen_prompt = PromptTemplate.from_template(data_gen_template)

generated_adverts = llm(data_gen_prompt.format(num_ads=num_ads, advert_separator=advert_separator))

/Users/me/opt/miniconda3/envs/udacity_cnn_project/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:141: LangChainDeprecationWarning: The method `BaseLLM.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


In [7]:
print(generated_adverts)


1. Advert #1: Cozy Home in the Heart of Suburbia
Neighborhood: Maplewood, New Jersey
Style: Colonial
Rooms: 6
Bedrooms: 3
Bathrooms: 2
House Size: 1,500 sqft
Price: $350,000
Real Estate Description: Welcome to your new home in the charming neighborhood of Maplewood. This colonial-style house boasts 3 spacious bedrooms, 2 full bathrooms, and a cozy living room perfect for family gatherings. With a generous 1,500 sqft of living space, this home is perfect for a growing family. Enjoy the convenience of a garage and a beautiful garden for outdoor activities. Don't miss out on this opportunity to own a piece of Maplewood for only $350,000!
Neighborhood Description: Maplewood is a family-friendly neighborhood known for its tree-lined streets, top-rated schools, and close-knit community. With easy access to major highways and public transportation, you'll have the best of both worlds - a peaceful suburban lifestyle with the bustling city just a short commute away.

2. Advert #2: Modern Townh

In [8]:
rows = generated_adverts.strip().split("\n")
for row in rows:
    print(row)

1. Advert #1: Cozy Home in the Heart of Suburbia
Neighborhood: Maplewood, New Jersey
Style: Colonial
Rooms: 6
Bedrooms: 3
Bathrooms: 2
House Size: 1,500 sqft
Price: $350,000
Real Estate Description: Welcome to your new home in the charming neighborhood of Maplewood. This colonial-style house boasts 3 spacious bedrooms, 2 full bathrooms, and a cozy living room perfect for family gatherings. With a generous 1,500 sqft of living space, this home is perfect for a growing family. Enjoy the convenience of a garage and a beautiful garden for outdoor activities. Don't miss out on this opportunity to own a piece of Maplewood for only $350,000!
Neighborhood Description: Maplewood is a family-friendly neighborhood known for its tree-lined streets, top-rated schools, and close-knit community. With easy access to major highways and public transportation, you'll have the best of both worlds - a peaceful suburban lifestyle with the bustling city just a short commute away.

2. Advert #2: Modern Townho

In [9]:
def parse_advert(str_advert: str) -> dict:
    advert = {}
    features = [
        "Index",
        "Neighborhood",
        "Style",
        "Rooms",
        "Bedrooms",
        "Bathrooms",
        "House Size",
        "Price",
        "Real Estate Description",
        "Neighborhood Description"
        ]
    regex_features = [r"^.*Advert #(\d+):.*$"]
    regex_features += ([re.compile(f"^{str}:\s*(.*)$") for str in features[1:]])
    
    split = str_advert.split("\n")
    for row in split:
        for feature, regex_feature in zip(features, regex_features):
            result = re.match(regex_feature, row)
            if result:
                advert[feature] = result.group(1)
                break

    #advert["Advert text"] = "\n".join(split[1:])
    return advert
    

In [10]:
generated_adverts_list = [ads.strip() for ads in generated_adverts.split(advert_separator)]
print(generated_adverts_list[0])
parse_advert(generated_adverts_list[0].strip())

1. Advert #1: Cozy Home in the Heart of Suburbia
Neighborhood: Maplewood, New Jersey
Style: Colonial
Rooms: 6
Bedrooms: 3
Bathrooms: 2
House Size: 1,500 sqft
Price: $350,000
Real Estate Description: Welcome to your new home in the charming neighborhood of Maplewood. This colonial-style house boasts 3 spacious bedrooms, 2 full bathrooms, and a cozy living room perfect for family gatherings. With a generous 1,500 sqft of living space, this home is perfect for a growing family. Enjoy the convenience of a garage and a beautiful garden for outdoor activities. Don't miss out on this opportunity to own a piece of Maplewood for only $350,000!
Neighborhood Description: Maplewood is a family-friendly neighborhood known for its tree-lined streets, top-rated schools, and close-knit community. With easy access to major highways and public transportation, you'll have the best of both worlds - a peaceful suburban lifestyle with the bustling city just a short commute away.


{'Index': '1',
 'Neighborhood': 'Maplewood, New Jersey',
 'Style': 'Colonial',
 'Rooms': '6',
 'Bedrooms': '3',
 'Bathrooms': '2',
 'House Size': '1,500 sqft',
 'Price': '$350,000',
 'Real Estate Description': "Welcome to your new home in the charming neighborhood of Maplewood. This colonial-style house boasts 3 spacious bedrooms, 2 full bathrooms, and a cozy living room perfect for family gatherings. With a generous 1,500 sqft of living space, this home is perfect for a growing family. Enjoy the convenience of a garage and a beautiful garden for outdoor activities. Don't miss out on this opportunity to own a piece of Maplewood for only $350,000!",
 'Neighborhood Description': "Maplewood is a family-friendly neighborhood known for its tree-lined streets, top-rated schools, and close-knit community. With easy access to major highways and public transportation, you'll have the best of both worlds - a peaceful suburban lifestyle with the bustling city just a short commute away."}

In [11]:
print(len(generated_adverts_list))

11


In [12]:
import json

In [13]:
generated_adverts_json = [parse_advert(advert.strip()) for advert in generated_adverts_list if advert]
for adv in generated_adverts_json:
    print(json.dumps(adv, indent=2))

{
  "Index": "1",
  "Neighborhood": "Maplewood, New Jersey",
  "Style": "Colonial",
  "Rooms": "6",
  "Bedrooms": "3",
  "Bathrooms": "2",
  "House Size": "1,500 sqft",
  "Price": "$350,000",
  "Real Estate Description": "Welcome to your new home in the charming neighborhood of Maplewood. This colonial-style house boasts 3 spacious bedrooms, 2 full bathrooms, and a cozy living room perfect for family gatherings. With a generous 1,500 sqft of living space, this home is perfect for a growing family. Enjoy the convenience of a garage and a beautiful garden for outdoor activities. Don't miss out on this opportunity to own a piece of Maplewood for only $350,000!",
  "Neighborhood Description": "Maplewood is a family-friendly neighborhood known for its tree-lined streets, top-rated schools, and close-knit community. With easy access to major highways and public transportation, you'll have the best of both worlds - a peaceful suburban lifestyle with the bustling city just a short commute away

In [16]:
filename = "generated_adverts_a.jsonl"
with open(filename, "w") as save_file:
    for generated_advert_json in generated_adverts_json:
        json.dump(generated_advert_json, save_file)
        save_file.write('\n')
save_file.close()

In [15]:
with open(filename, "r") as file:
    for line in file:
        data_entry = json.loads(line)
        # Process each data_entry as a Python dict
        print(data_entry)

{'Index': '1', 'Neighborhood': 'Maplewood, New Jersey', 'Style': 'Colonial', 'Rooms': '6', 'Bedrooms': '3', 'Bathrooms': '2', 'House Size': '1,500 sqft', 'Price': '$350,000', 'Real Estate Description': "Welcome to your new home in the charming neighborhood of Maplewood. This colonial-style house boasts 3 spacious bedrooms, 2 full bathrooms, and a cozy living room perfect for family gatherings. With a generous 1,500 sqft of living space, this home is perfect for a growing family. Enjoy the convenience of a garage and a beautiful garden for outdoor activities. Don't miss out on this opportunity to own a piece of Maplewood for only $350,000!", 'Neighborhood Description': "Maplewood is a family-friendly neighborhood known for its tree-lined streets, top-rated schools, and close-knit community. With easy access to major highways and public transportation, you'll have the best of both worlds - a peaceful suburban lifestyle with the bustling city just a short commute away."}
{'Index': '2', 'N